# Cross-service querying with Kusto Notebook in Azure Data Studio

This example uses help.kusto.windows.net StormEvents data and joining it with my own private Azure Data Explorer cluster myADXcluster that is in the West Us region. 

The goal is to get the number of Storm Event Counts for the top 3 most populated State. 

This example exhibits using `cluster('cluster.region').databse('dbname')` syntax.


First, change the Kernel to **Kusto** and Attach to **help.kusto.windows.net** and set the database to **Samples**. Run the following query to ensure that it works. 

In [1]:
StormEvents 
| summarize EventCount = count() by State
| take 3

Commands completed successfully.

Total execution time: 00:00:00.176

State,EventCount
ATLANTIC SOUTH,193
FLORIDA,1042
GEORGIA,1983


The AAD account that I used earlier to connect to **help.kusto.windows.net**, should also allow me to connect to my **myADXcluster** cluster. 

> **Tip**: since I have multiple accounts, I need to ensure that I'm logging in to the right AAD account :)


The following query allows me to perform cross cluster querying with joins. This retrieves the number of Storm Event Counts for the top 3 most populated State. 

In [2]:
let stormSummarized = StormEvents 
| summarize EventCount = count() by State = toupper(State);
cluster('myADXcluster.westus').database('mykustodb').StatePopulation
| project State = toupper(State), Population
| join (stormSummarized) on State
| project State, Population, EventCount
| sort by Population 
| take 3



Commands completed successfully.

Total execution time: 00:00:00.961

State,Population,EventCount
CALIFORNIA,39512223,898
TEXAS,28995881,4701
FLORIDA,21477737,1042


## Cross Service query (without joining)

Yes, it is possible to query Azure Log Analytics workspace while you are actively connected to a help ADX cluster or any other ADX cluster that you have access to. 

> **Note**: As of Azure Data Studio v1.25.1, the Native KQL (KQL Extension) does not support connecting to Azure Log Analytics workspace natively. This is a workaround for that. 


In [3]:
cluster('myADXcluster.westus').database('mykustodb').StatePopulation
| sort by Population 
| take 10

Commands completed successfully.

Total execution time: 00:00:00.172

State,Population
California,39512223
Texas,28995881
Florida,21477737
New York,19453561
Pennsylvania,12801989
Illinois,12671821
Ohio,11689100
Georgia,10617423
North Carolina,10488084
Michigan,9986857
